In [4]:
# Marc Sorrentino 2024-08
# Model development for realtime spectral fitting and embedding

# Installations:

In [5]:
# !pip install torch
# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install torchinfo
# !pip install torchsummary
# !pip install scipy
# !pip install h5py

# IMPORTS

In [1]:
import sys
sys.path.append('/home/user/Code_Repo/n-CORTEx/NeuralNetworks/RTSpec')
# sys.path.append('/home/user/Code_Repo/n-CORTEx/NeuralNetworks/RTSpec/Models')
from Models.UTime import USpec, Specformer, Specformer2, Specformer3
from Models.FCN import specFCN, specFCN2, specFCN3, specFCN4, specFCN6, specFCN15
from Models.LSTM import specLSTM, LSTM75, biLSTM50, resBiLSTM50
from Assembly.SpecsDataset import SpecsDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
# from torchsummary import summary
import numpy as np
import math
import glob
from count_parameters import count_parameters
from train import train 
from test_model import test_model
import time
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR

# ASSEMBLY PARAMETERS

In [2]:
import os
# Define assembly parameters
params = {}
params['startEpoch'] = 0
params['endEpoch'] = 180
params['local_rank'] = 0
# params['modelName'] = "USpec"
params['modelName'] = "biLSTM50"
params['numWorkers'] = 4
params['batchSize'] = 39
# params['batchSize'] = 100
params['learningRate'] = 0.0005
# params['learningRate'] = 0.0002
params['dropoutRate'] = 0.25
inventoryDir = "/home/user/nCORTEx_local/Project_Neuromodulation-for-Pain/Experiments/JOLT/Data/FTR/TRAIN/RTSpec/Inventory"
dataDir = "/home/user/nCORTEx_local/Project_Neuromodulation-for-Pain/Experiments/JOLT/Data/FTR/TRAIN/RTSpec/Dataset"
checkpointDir = os.path.join(inventoryDir,params['modelName'])
os.makedirs(checkpointDir, exist_ok=True)
# count the number of 'training sessions' for a given model
numModelItrs = sum(1 for item in os.listdir(checkpointDir) if os.path.isdir(os.path.join(checkpointDir, item)))
params['checkpoint_dir']=os.path.join("/home/user/nCORTEx_local/Project_Neuromodulation-for-Pain/Experiments/JOLT/Data/FTR/TRAIN/RTSpec/Inventory",params['modelName'],("T"+str(numModelItrs).rjust(2,'0')))    
params['log_dir']=params['checkpoint_dir']
params['dataDir'] = dataDir


# SETUP

In [ ]:
# Fold settings (Cross Validation)
trainFolds = [1,2,3]
validFold = [4]
testFold = [5]
# Dataset instantiation
specsData_train = SpecsDataset(dataDir, trainFolds)
specsData_valid = SpecsDataset(dataDir, validFold)
specsData_test = SpecsDataset(dataDir, testFold)
# Dataloader instantiation
trainLoader = DataLoader(specsData_train, shuffle=True, num_workers=params['numWorkers'], batch_size=params['batchSize'], pin_memory=False)
validLoader = DataLoader(specsData_valid, shuffle=True, num_workers=params['numWorkers'], batch_size=params['batchSize'], pin_memory=False)
testloader = DataLoader(specsData_valid, shuffle=True, num_workers=params['numWorkers'], batch_size=params['batchSize'], pin_memory=False)
# Model instantiation
# model = USpec()
# model = specFCN(dropout_rate=params['dropoutRate'])
model = biLSTM50(dropout_prob=0.25)
model = model.cuda()
modelSize = count_parameters(model)
print(modelSize)
# Test pass (size validation)
# # Create a sample input tensor with shape (batch_size, channels, length)
input_tensor = torch.randn(1, 1, 196)
# # Run the forward pass and print the shapes
start=time.time()
output = model(input_tensor.cuda())
stop=time.time()
lat=stop-start
print("Pass Latency: " + str(lat))
print(output)
# Model Summary
print(summary(model, input_size=(1, 1, 196)))
# Optimizer instantiation
# optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay, amsgrad=args.amsgrad)
optimizer = torch.optim.Adam(model.parameters(), lr=params['learningRate'], weight_decay=0.0001, amsgrad=True)
# optimizer = torch.optim.AdamW(model.parameters(), lr=params['learningRate'], weight_decay=0.0001, amsgrad=True)
scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=0.0001)
criterion = nn.MSELoss(reduction='sum')

In [ ]:
# load_dir = '/home/user/nCORTEx_local/Project_Neuromodulation-for-Pain/Experiments/JOLT/Data/FTR/TRAIN/RTSpec/Inventory/specFCN4/T16/model_specFCN4_epoch_4_val_loss_44782.20130729675.pth'
# # load_dir = '/home/user/nCORTEx_local/Project_Neuromodulation-for-Pain/Experiments/JOLT/Data/FTR/TRAIN/RTSpec/Inventory/specFCN4/T04/model_specFCN4_epoch_98_val_loss_7545.2792888810145.pth'
# checkpoint = torch.load(load_dir, map_location=torch.device('cuda'))
# model.load_state_dict(checkpoint['state_dict'])
# model.cuda()

# TRAINING

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
# optimizer = torch.optim.Adam(model.parameters(), lr=params['learningRate'], weight_decay=0.0001, amsgrad=True)

In [ ]:
if torch.cuda.is_available():
    # Model Training
    model = train(params, model, trainLoader, validLoader, criterion, optimizer, scheduler)

# TESTING

# CROSS-VALIDATION

In [ ]:
import numpy as np

numFolds = 5
CV_r2_off_mean = []
CV_r2_off_SE = []
CV_r2_exp_mean = []
CV_r2_exp_SE = []
CV_r2_CF_mean = []
CV_r2_CF_SE = []
CV_r2_BW_mean = []
CV_r2_BW_SE = []
CV_r2_PW_mean = []
CV_r2_PW_SE = []
CV_resid_off_mean = []
CV_resid_off_SE = []
CV_resid_exp_mean = []
CV_resid_exp_SE = []
CV_resid_CF_mean = []
CV_resid_CF_SE = []
CV_resid_BW_mean = []
CV_resid_BW_SE = []
CV_resid_PW_mean = []
CV_resid_PW_SE = []
CV_nrmse_off_mean = []
CV_nrmse_off_SE = []
CV_nrmse_exp_mean = []
CV_nrmse_exp_SE = []
CV_nrmse_CF_mean = []
CV_nrmse_CF_SE = []
CV_nrmse_BW_mean = []
CV_nrmse_BW_SE = []
CV_nrmse_PW_mean = []
CV_nrmse_PW_SE = []
CV_r2_total_mean = []
CV_r2_total_SE = []
CV_resid_total_mean = []
CV_resid_total_SE = []
CV_nrmse_total_mean = []
CV_nrmse_total_SE = []

# model = specFCN6()
# # load_dir = '/home/user/nCORTEx_local/Project_Neuromodulation-for-Pain/Experiments/JOLT/Data/FTR/TRAIN/RTSpec/Inventory/specFCN4/T07/model_specFCN4_epoch_236_val_loss_7290.956895816201.pth'
# # load_dir = '/home/user/nCORTEx_local/Project_Neuromodulation-for-Pain/Experiments/JOLT/Data/FTR/TRAIN/RTSpec/Inventory/specFCN4/T04/model_specFCN4_epoch_98_val_loss_7545.2792888810145.pth'
# load_dir = '/home/user/nCORTEx_local/Project_Neuromodulation-for-Pain/Experiments/JOLT/Data/FTR/TRAIN/RTSpec/Inventory/specFCN6/T02/model_specFCN6_epoch_165_val_loss_18783.800958393895.pth'
# checkpoint = torch.load(load_dir, map_location=torch.device('cuda'))
# model.load_state_dict(checkpoint['state_dict'])
# model.cuda()

for i in range(numFolds):
    print(i)
    print("Fold: " + str(i+1))    
    print("TRAINING / VALIDATION: ")    
    
    # Define the folds
    all_folds = list(range(numFolds))
    all_folds = [fold + 1 for fold in all_folds]
    m = i + 1
    # Define the validation fold
    validFold = [m]
    
    # Define the test fold
    if m == numFolds:
        testFold = [1]
    else:
        testFold = [(m + 1) % numFolds]
    
    # Define the training folds
    trainFolds = [fold for fold in all_folds if fold not in validFold and fold not in testFold]
    print("Train Folds: " + str(trainFolds))
    print("Valid Fold: " + str(validFold))
    print("Test Fold: " + str(testFold))
    # Model instantiation
    model = biLSTM50(dropout_prob=0.25)
    model = model.cuda()
    # optimizer setup
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learningRate'], weight_decay=0.0001, amsgrad=True)
    # optimizer = torch.optim.AdamW(model.parameters(), lr=params['learningRate'], weight_decay=0.0001, amsgrad=True)
    scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=0.0001)
    criterion = nn.MSELoss(reduction='sum')
    # Dataset instantiation
    specsData_train = SpecsDataset(dataDir, trainFolds)
    specsData_valid = SpecsDataset(dataDir, validFold)
    specsData_test = SpecsDataset(dataDir, testFold)
    # Dataloader instantiation
    trainLoader = DataLoader(specsData_train, shuffle=True, num_workers=params['numWorkers'], batch_size=params['batchSize'], pin_memory=False)
    validLoader = DataLoader(specsData_valid, shuffle=False, num_workers=params['numWorkers'], batch_size=params['batchSize'], pin_memory=False)
    testLoader = DataLoader(specsData_valid, shuffle=False, num_workers=params['numWorkers'], batch_size=params['batchSize'], pin_memory=False)
    # Begin training
    model = train(params, model, trainLoader, validLoader, criterion, optimizer, scheduler)
    # Begin testing
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learningRate'], weight_decay=0.0001, amsgrad=True)
    # optimizer = torch.optim.AdamW(model.parameters(), lr=params['learningRate'], weight_decay=0.0001, amsgrad=True)
    # scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=0.0001)
    criterion = nn.MSELoss(reduction='sum')
    print("TESTING: ")
    test_losses, mean_resid_off, SE_resid_off, mean_resid_exp, SE_resid_exp, mean_resid_CF, SE_resid_CF, mean_resid_BW, SE_resid_BW, mean_resid_PW, SE_resid_PW, mean_nrmse_off, SE_nrmse_off, mean_nrmse_exp, SE_nrmse_exp, mean_nrmse_CF, SE_nrmse_CF, mean_nrmse_BW, SE_nrmse_BW, mean_nrmse_PW, SE_nrmse_PW, mean_resid_total, mean_nrmse_total, SE_resid_total, SE_nrmse_total = test_model(params, model, testLoader, criterion)
    
    # CV_r2_off_mean.append(mean_r2_off)
    # CV_r2_off_SE.append(SE_r2_off)
    # CV_r2_exp_mean.append(mean_r2_exp)
    # CV_r2_exp_SE.append(SE_r2_exp)
    # CV_r2_CF_mean.append(mean_r2_CF)
    # CV_r2_CF_SE.append(SE_r2_CF)
    # CV_r2_BW_mean.append(mean_r2_BW)
    # CV_r2_BW_SE.append(SE_r2_BW)
    # CV_r2_PW_mean.append(mean_r2_PW)
    # CV_r2_PW_SE.append(SE_r2_PW)

    CV_resid_off_mean.append(mean_resid_off)
    CV_resid_off_SE.append(SE_resid_off)
    CV_resid_exp_mean.append(mean_resid_exp)
    CV_resid_exp_SE.append(SE_resid_exp)    
    CV_resid_CF_mean.append(mean_resid_CF)
    CV_resid_CF_SE.append(SE_resid_CF)    
    CV_resid_BW_mean.append(mean_resid_BW)
    CV_resid_BW_SE.append(SE_resid_BW)
    CV_resid_PW_mean.append(mean_resid_PW)
    CV_resid_PW_SE.append(SE_resid_PW)

    CV_nrmse_off_mean.append(mean_nrmse_off)
    CV_nrmse_off_SE.append(SE_nrmse_off)
    CV_nrmse_exp_mean.append(mean_nrmse_exp)
    CV_nrmse_exp_SE.append(SE_nrmse_exp)
    CV_nrmse_CF_mean.append(mean_nrmse_CF)
    CV_nrmse_CF_SE.append(SE_nrmse_CF)
    CV_nrmse_BW_mean.append(mean_nrmse_BW)
    CV_nrmse_BW_SE.append(SE_nrmse_BW)
    CV_nrmse_PW_mean.append(mean_nrmse_PW)
    CV_nrmse_PW_SE.append(SE_nrmse_PW)
    
    CV_resid_total_mean.append(mean_resid_total)
    CV_resid_total_SE.append(SE_resid_total)
    CV_nrmse_total_mean.append(mean_nrmse_total)
    CV_nrmse_total_SE.append(SE_nrmse_total)        

# CV_r2_off_mean = torch.mean(CV_r2_off_mean)
# CV_r2_off_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_r2_off_SE)))) / numFolds).item()
# CV_r2_exp_mean = torch.mean(torch.tensor(CV_r2_exp_mean)).item()
# CV_r2_exp_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_r2_exp_SE)))) / numFolds).item()
# CV_r2_CF_mean = torch.mean(torch.tensor(CV_r2_CF_mean)).item()
# CV_r2_CF_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_r2_CF_SE)))) / numFolds).item()
# CV_r2_BW_mean = torch.mean(torch.tensor(CV_r2_BW_mean)).item()
# CV_r2_BW_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_r2_BW_SE)))) / numFolds).item()
# CV_r2_PW_mean = torch.mean(torch.tensor(CV_r2_PW_mean)).item()
# CV_r2_PW_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_r2_PW_SE)))) / numFolds).item()

CV_resid_off_mean = torch.mean(torch.tensor(CV_resid_off_mean)).item()
CV_resid_off_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_resid_off_SE)))) / numFolds).item()
CV_resid_exp_mean = torch.mean(torch.tensor(CV_resid_exp_mean)).item()
CV_resid_exp_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_resid_exp_SE)))) / numFolds).item()
CV_resid_CF_mean = torch.mean(torch.tensor(CV_resid_CF_mean)).item()
CV_resid_CF_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_resid_CF_SE)))) / numFolds).item()
CV_resid_BW_mean = torch.mean(torch.tensor(CV_resid_BW_mean)).item()
CV_resid_BW_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_resid_BW_SE)))) / numFolds).item()
CV_resid_PW_mean = torch.mean(torch.tensor(CV_resid_PW_mean)).item()
CV_resid_PW_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_resid_PW_SE)))) / numFolds).item()

print("CV_resid_off_mean: " + str(CV_resid_off_mean))
print("CV_resid_exp_mean: " + str(CV_resid_exp_mean))
print("CV_resid_CF_mean: " + str(CV_resid_CF_mean))
print("CV_resid_BW_mean: " + str(CV_resid_BW_mean))
print("CV_resid_PW_mean: " + str(CV_resid_PW_mean))
print("CV_resid_total_mean: " + str(CV_resid_total_mean))

CV_nrmse_off_mean = torch.mean(torch.tensor(CV_nrmse_off_mean)).item()
CV_nrmse_off_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_nrmse_off_SE)))) / numFolds).item()
CV_nrmse_exp_mean = torch.mean(torch.tensor(CV_nrmse_exp_mean)).item()
CV_nrmse_exp_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_nrmse_exp_SE)))) / numFolds).item()
CV_nrmse_CF_mean = torch.mean(torch.tensor(CV_nrmse_CF_mean)).item()
CV_nrmse_CF_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_nrmse_CF_SE)))) / numFolds).item()
CV_nrmse_BW_mean = torch.mean(torch.tensor(CV_nrmse_BW_mean)).item()
CV_nrmse_BW_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_nrmse_BW_SE)))) / numFolds).item()
CV_nrmse_PW_mean = torch.mean(torch.tensor(CV_nrmse_PW_mean)).item()
CV_nrmse_PW_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_nrmse_PW_SE)))) / numFolds).item()

CV_r2_total_mean = torch.mean(torch.tensor(CV_r2_total_mean)).item()
CV_r2_total_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_r2_total_SE)))) / numFolds).item()
CV_resid_total_mean = torch.mean(torch.tensor(CV_resid_total_mean)).item()
CV_resid_total_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_resid_total_SE)))) / numFolds).item()
CV_nrmse_total_mean = torch.mean(torch.tensor(CV_nrmse_total_mean)).item()
CV_nrmse_total_SE = (torch.sqrt(torch.sum(torch.square(torch.tensor(CV_nrmse_total_SE)))) / numFolds).item()

print("CV_nrmse_off_mean: " + str(CV_nrmse_off_mean))
print("CV_nrmse_exp_mean: " + str(CV_nrmse_exp_mean))
print("CV_nrmse_CF_mean: " + str(CV_nrmse_CF_mean))
print("CV_nrmse_BW_mean: " + str(CV_nrmse_BW_mean))
print("CV_nrmse_PW_mean: " + str(CV_nrmse_PW_mean))
print("CV_nrmse_total_mean: " + str(CV_nrmse_total_mean))

# PLOT RESULTS
import matplotlib.pyplot as plt

# Data preparation
categories = ['OFF', 'EXP', 'CF', 'BW', 'PW', 'PSD']
mean_values = [
    100 - CV_nrmse_off_mean,
    100 - CV_nrmse_exp_mean,
    100 - CV_nrmse_CF_mean,
    100 - CV_nrmse_BW_mean,
    100 - CV_nrmse_PW_mean,
    100 - CV_nrmse_total_mean
]
se_values = [
    CV_nrmse_off_SE,
    CV_nrmse_exp_SE,
    CV_nrmse_CF_SE,
    CV_nrmse_BW_SE,
    CV_nrmse_PW_SE,
    CV_nrmse_total_SE
]
# Bar plot configuration
bar_width = 0.5
x = np.arange(len(categories))

# Plotting
fig, ax = plt.subplots(figsize=(12, 6))
fig.patch.set_facecolor('#98b69e')
ax.set_facecolor('#98b69e')

# Plotting accuracy scores
color = '#c0ffd9'
ax.bar(x, mean_values, bar_width, yerr=se_values, label='Accuracy (100 - NRMSE)', capsize=5, color=color)

# Customizing the plot
ax.set_xlabel('Parameters', fontsize=16, color='#00240d')
ax.set_ylabel('Accuracy (%)', fontsize=16, color='#00240d')
ax.set_title('NRMSE-derived Accuracy', fontsize=16, color='#00240d')
ax.set_xticks(x)
ax.set_xticklabels(categories, fontsize=16, color='#00240d')
ax.legend(facecolor='#98b69e', edgecolor='#00240d', labelcolor='#00240d', fontsize=16)
ax.tick_params(axis='y', colors='#00240d', labelsize=16)
ax.tick_params(axis='x', colors='#00240d', labelsize=16)

# Change bounding box color to '#00240d'
for spine in ax.spines.values():
    spine.set_edgecolor('#00240d')

plt.tight_layout()
plt.show()
save_name = os.path.join(params['checkpoint_dir'],('performance_rates_'+params['modelName']+'.png'))
plt.savefig(save_name)


# LATENCY TESTING

In [ ]:
# PLOT RESULTS
import matplotlib.pyplot as plt

# Data preparation
categories = ['OFF', 'EXP', 'CF', 'BW', 'PW', 'Total']
mean_values = [
    [CV_resid_off_mean, CV_nrmse_off_mean],
    [CV_resid_exp_mean, CV_nrmse_exp_mean],
    [CV_resid_CF_mean, CV_nrmse_CF_mean],
    [CV_resid_BW_mean, CV_nrmse_BW_mean],
    [CV_resid_PW_mean, CV_nrmse_PW_mean],
    [CV_resid_total_mean, CV_nrmse_total_mean]
]
se_values = [
    [CV_resid_off_SE, CV_nrmse_off_SE],
    [CV_resid_exp_SE, CV_nrmse_exp_SE],
    [CV_resid_CF_SE, CV_nrmse_CF_SE],
    [CV_resid_BW_SE, CV_nrmse_BW_SE],
    [CV_resid_PW_SE, CV_nrmse_PW_SE],
    [CV_resid_total_SE, CV_nrmse_total_SE]
]

# Bar plot configuration
bar_width = 0.3
x = np.arange(len(categories))

# Plotting
fig, ax = plt.subplots(figsize=(12, 6))

for i in range(2):  # For resid, nrmse
    means = [mean_values[j][i] for j in range(len(categories))]
    ses = [se_values[j][i] for j in range(len(categories))]
    ax.bar(x + i * bar_width, means, bar_width, yerr=ses, label=['Resid', 'NRMSE'][i], capsize=5)

# Customizing the plot
ax.set_xlabel('Categories')
ax.set_ylabel('Values')
ax.set_title('Mean and SE for Different Metrics')
ax.set_xticks(x + bar_width / 2)
ax.set_xticklabels(categories)
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# PLOT RESULTS
import matplotlib.pyplot as plt

# Data preparation
categories = ['OFF', 'EXP', 'CF', 'BW', 'PW', 'PSD']
mean_values = [
    100 - CV_nrmse_off_mean,
    100 - CV_nrmse_exp_mean,
    100 - CV_nrmse_CF_mean,
    100 - CV_nrmse_BW_mean,
    100 - CV_nrmse_PW_mean,
    100 - CV_nrmse_total_mean
]
se_values = [
    CV_nrmse_off_SE,
    CV_nrmse_exp_SE,
    CV_nrmse_CF_SE,
    CV_nrmse_BW_SE,
    CV_nrmse_PW_SE,
    CV_nrmse_total_SE
]
# Bar plot configuration
bar_width = 0.5
x = np.arange(len(categories))

# Plotting
fig, ax = plt.subplots(figsize=(12, 6))
fig.patch.set_facecolor('#98b69e')
ax.set_facecolor('#98b69e')

# Plotting accuracy scores
color = '#c0ffd9'
ax.bar(x, mean_values, bar_width, yerr=se_values, label='Accuracy (100 - NRMSE)', capsize=5, color=color)

# Customizing the plot
ax.set_xlabel('Parameters', fontsize=16, color='#00240d')
ax.set_ylabel('Accuracy (%)', fontsize=16, color='#00240d')
ax.set_title('NRMSE-derived Accuracy', fontsize=16, color='#00240d')
ax.set_xticks(x)
ax.set_xticklabels(categories, fontsize=16, color='#00240d')
ax.legend(facecolor='#98b69e', edgecolor='#00240d', labelcolor='#00240d', fontsize=16)
ax.tick_params(axis='y', colors='#00240d', labelsize=16)
ax.tick_params(axis='x', colors='#00240d', labelsize=16)

# Change bounding box color to '#00240d'
for spine in ax.spines.values():
    spine.set_edgecolor('#00240d')

plt.tight_layout()
plt.show()
save_name = os.path.join(params['checkpoint_dir'],('performance_rates_'+params['modelName']+'.png'))
plt.savefig(save_name)